In [0]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F

In [2]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

In [0]:
#Загружаем данные из файла.
with open('dostoevsky.txt', 'r') as f:
  text = f.read()

In [4]:
#Выведем 100 первых символов текста.
text[:100]

'Федор Михайлович Достоевский\nБедные люди\nОх уж эти мне сказочники! Нет чтобы написать что-нибудь пол'

In [0]:
#Закодируем символы в целые числа
chars = tuple(set(text))
int2char = dict(enumerate(chars))
char2int = {ch: ii for ii, ch in int2char.items()}

encoded = np.array([char2int[ch] for ch in text])

In [6]:
#Выведем те же 100 символов, только в численном представлении.
encoded[:100] 

array([ 16,   7, 116,  73,  76,  88,  84, 127, 131, 118,  42, 120,  73,
        34, 127,  64,  88, 104,  73, 124, 128,  73,   7,  34, 124,  19,
       127,  42,  58, 114,   7, 116,  23,  53,   7,  88, 120,  97, 116,
       127,  58, 105, 131,  88,  40,  32,  88,  78, 128, 127,  88, 102,
        23,   7,  88, 124,  19, 118, 132,  73,  64,  23, 127,  19, 127,
       107,  88, 129,   7, 128,  88,  64, 128,  73,  65,  53,  88,  23,
       118,  54, 127, 124, 118, 128, 117,  88,  64, 128,  73, 101,  23,
       127,  65,  40, 116, 117,  88,  54,  73, 120])

In [0]:
#Используем прямое кодирование для представления букв.
def one_hot_encode(arr, n_labels):
    one_hot = np.zeros((arr.size, n_labels), dtype=np.float32)
    one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1.
    one_hot = one_hot.reshape((*arr.shape, n_labels))

    return one_hot

In [8]:
#Посмотрим как работает функция,написанная выше.
test_seq = np.array([3, 5, 1])
one_hot = one_hot_encode(test_seq, 8)

print(one_hot)

[[0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0.]]


In [0]:
#Генерация мини-батчей
def get_batches(arr, batch_size, seq_length):

    batch_size_total = batch_size * seq_length
    n_batches = len(arr) // batch_size_total

    arr = arr[:n_batches*batch_size_total]

    arr = arr.reshape((batch_size, -1))

    for n in range(0, arr.shape[1], seq_length):
        x = arr[:, n:n+seq_length]  
        y = np.zeros_like(x)
        try:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, n+seq_length]
        except IndexError:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, 0]
        yield x, y

In [10]:
#Протестируем генерацию мини-батчей.
batches = get_batches(encoded, 8, 50)
x, y = next(batches)

print('x\n', x[:10, :10])
print('\ny\n', y[:10, :10])
#Посмотрим на 6 строчку ноутбука и убедимся в правильности.

x
 [[ 16   7 116  73  76  88  84 127 131 118]
 [120  92  88  40 120  53  65 118  60 124]
 [ 88  34  88  54  76 127  34  53  64  23]
 [124 117  88  73 124 128 118  34 118 128]
 [116 117  88  23 118 102   7  76   7  23]
 [ 88 124   7  62  73 116  23  60  88 116]
 [ 23   7  88 124  73  34 124   7 102  88]
 [127  88  54  73  54 118 120  73  92  88]]

y
 [[  7 116  73  76  88  84 127 131 118  42]
 [ 92  88  40 120  53  65 118  60 124 117]
 [ 34  88  54  76 127  34  53  64  23  73]
 [117  88  73 124 128 118  34 118 128 117]
 [117  88  23 118 102   7  76   7  23 127]
 [124   7  62  73 116  23  60  88 116  73]
 [  7  88 124  73  34 124   7 102  88   7]
 [ 88  54  73  54 118 120  73  92  88 118]]


In [11]:
train_on_gpu = torch.cuda.is_available()
if(train_on_gpu):
  print('Training on GPU!')
else:
  print('Training on CPU')

Training on GPU!


In [0]:
class CharRNN(nn.Module):

    def __init__(self, tokens, n_hidden=256, n_layers=2,
                               drop_prob=0.5, lr=0.001):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr

        # Создаем символьные словари
        self.chars = tokens
        self.int2char = dict(enumerate(self.chars))
        self.char2int = {ch: ii for ii, ch in self.int2char.items()}

        # Определяем слои модели.
        self.lstm = nn.LSTM(len(self.chars), n_hidden, n_layers,
                           dropout=drop_prob, batch_first=True)

        self.dropout = nn.Dropout(drop_prob)

        # Определяем full-connection слой 
        self.fc = nn.Linear(n_hidden, len(self.chars))

    def forward(self, x, hidden):
        r_output, hidden = self.lstm(x, hidden)
        out = self.dropout(r_output)
        out = out.contiguous().view(-1, self.n_hidden)
        out = self.fc(out)

        return out, hidden

    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
                      weight.new(self.n_layers, batch_size, self.n_hidden).zero_())

        return hidden

In [0]:
#Обучение сети.
def train(net, data, epochs=10, batch_size=10, seq_length=50, lr=0.001, clip=5, val_frac=0.1, print_every=10):
    
    net.train()

    opt = torch.optim.RMSprop(net.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()

    # Создаем обучающие и валидационные данные.
    val_idx = int(len(data)*(1-val_frac))
    data, val_data = data[:val_idx], data[val_idx:]

    if(train_on_gpu):
        net.cuda()

    counter = 0
    n_chars = len(net.chars)
    for e in range(epochs):
        h = net.init_hidden(batch_size)

        for x, y in get_batches(data, batch_size, seq_length):
            counter += 1

            x = one_hot_encode(x, n_chars)
            inputs, targets = torch.from_numpy(x), torch.from_numpy(y)

            if(train_on_gpu):
                inputs, targets = inputs.cuda(), targets.cuda()

            h = tuple([each.data for each in h])

            net.zero_grad()

            output, h = net(inputs, h)

            loss = criterion(output, targets.view(batch_size*seq_length).long())
            loss.backward()
            nn.utils.clip_grad_norm_(net.parameters(), clip)
            opt.step()

            if counter % print_every == 0:
                val_h = net.init_hidden(batch_size)
                val_losses = []
                net.eval()
                for x, y in get_batches(val_data, batch_size, seq_length):
                    x = one_hot_encode(x, n_chars)
                    x, y = torch.from_numpy(x), torch.from_numpy(y)

                    val_h = tuple([each.data for each in val_h])

                    inputs, targets = x, y
                    if(train_on_gpu):
                        inputs, targets = inputs.cuda(), targets.cuda()

                    output, val_h = net(inputs, val_h)
                    val_loss = criterion(output, targets.view(batch_size*seq_length).long())

                    val_losses.append(val_loss.item())

                net.train()

                print("Epoch: {}/{}...".format(e+1, epochs),
                      "Step: {}...".format(counter),
                      "Loss: {:.4f}...".format(loss.item()),
                      "Val Loss: {:.4f}".format(np.mean(val_losses)))

In [14]:
n_hidden=512
n_layers=2

net = CharRNN(chars, n_hidden, n_layers)
print(net)

CharRNN(
  (lstm): LSTM(133, 512, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=512, out_features=133, bias=True)
)


In [15]:
batch_size = 128
seq_length = 100
n_epochs = 40 
#Обучаем модель.
train(net, encoded, epochs=n_epochs, batch_size=batch_size, seq_length=seq_length, lr=0.001, print_every=10)

Epoch: 1/40... Step: 10... Loss: 3.2915... Val Loss: 3.2856
Epoch: 1/40... Step: 20... Loss: 3.2418... Val Loss: 3.2879
Epoch: 1/40... Step: 30... Loss: 3.2531... Val Loss: 3.2871
Epoch: 2/40... Step: 40... Loss: 3.2675... Val Loss: 3.2863
Epoch: 2/40... Step: 50... Loss: 3.2211... Val Loss: 3.2850
Epoch: 2/40... Step: 60... Loss: 3.2454... Val Loss: 3.2895
Epoch: 3/40... Step: 70... Loss: 3.2442... Val Loss: 3.2791
Epoch: 3/40... Step: 80... Loss: 3.2138... Val Loss: 3.2833
Epoch: 3/40... Step: 90... Loss: 3.2198... Val Loss: 3.2719
Epoch: 4/40... Step: 100... Loss: 3.2056... Val Loss: 3.2397
Epoch: 4/40... Step: 110... Loss: 3.1455... Val Loss: 3.2008
Epoch: 4/40... Step: 120... Loss: 3.1053... Val Loss: 3.1561
Epoch: 5/40... Step: 130... Loss: 3.0847... Val Loss: 3.1221
Epoch: 5/40... Step: 140... Loss: 3.0365... Val Loss: 3.0664
Epoch: 5/40... Step: 150... Loss: 2.9673... Val Loss: 3.0168
Epoch: 6/40... Step: 160... Loss: 2.9399... Val Loss: 2.9751
Epoch: 6/40... Step: 170... Loss:

In [0]:
#Сохраним нашу модель
model_name = 'rnn_x_epoch.net'
checkpoint = {
    'n_hidden': net.n_hidden,
    'n_layers': net.n_layers,
    'state_dict': net.state_dict(),
    'tokens': net.chars}

with open(model_name, 'wb') as f:
  torch.save(checkpoint, f)

In [17]:
def predict(net, char, h=None, top_k=None):
  x = np.array([[net.char2int[char]]])
  x = one_hot_encode(x, len(net.chars))
  inputs = torch.from_numpy(x)
  if(train_on_gpu):
    inputs = inputs.cuda()

  h = tuple([each.data for each in h])
  out, h = net(inputs, h)
  p = F.softmax(out, dim=1).data
  if(train_on_gpu):
    p = p.cpu()
  if top_k is None:
    top_ch = np.arange(len(net.chars))
  else:
    p, top_ch = p.topk(top_k)
    top_ch = top_ch.numpy().squeeze()
  p = p.numpy().squeeze()
  char = np.random.choice(top_ch, p=p/p.sum())

  return net.int2char[char], h

def sample(net, size, prime='The', top_k=None):
  if(train_on_gpu):
    net.cuda()
  else:
    net.cpu()
  net.eval() 
  chars = [ch for ch in prime]
  h = net.init_hidden(1)

  for ch in prime:
    char, h = predict(net, ch, h, top_k=top_k)
  chars.append(char)

  for ii in range(size):
    char, h = predict(net, chars[-1], h, top_k=top_k)
    chars.append(char)

  return ''.join(chars)
  
print(sample(net, 1000, prime='Анна', top_k=5))

Анная; по меня столько стоит подудель вогороде свои и про стыл советить в сердце всем и придомилась с ним своех, что он был одна в какой-то в пода вам все трепутался, что вы только от слово, что все странной слышит, и про мое блогоромнее вословим, что он сама преждетель в плакное своим сердце моей, чего все-таки пристарных совенена и возмение, к посмотрите минула со даленеем своим солокним, и под нас все свете служает весьма, вся непогодном выходите, чтоб не следа то что-то испетивается, что вот в свою нашимали, когда мы слова, покраснываю, чтобы и не смирать нашем совсем неменника, несказное все на себя.
Милостительной друг мой! Вот я ваду и тут он из ничего; и вы покучайте, как и все подусинней моих драже и так селецины, что уже слезами старовой, и не вышел свидал меня изверениим весливанием плакая потому ставом слезы,
— стали потрясительное всего в тему. Ну, доставало, что не прочиться вашу сердце, на сегодня, пример в простила, что я сказал он носчилось вашу, и стыр и не смелнила о

In [18]:
#Загрузка модели.
with open('rnn_x_epoch.net', 'rb') as f:
  checkpoint = torch.load(f)

loaded = CharRNN(checkpoint['tokens'], n_hidden=checkpoint['n_hidden'], n_layers=checkpoint['n_layers'])
loaded.load_state_dict(checkpoint['state_dict'])

print(sample(loaded, 2000, top_k=5, prime="И он сказал"))

И он сказал с ним. Я письмо себе вспугнивает нуждо, и ни старом, что все вы старый мидуты столоки. Я не выстинный, в тратьером превестами, и поседели в кравеце с постовини, что в друг мой, стало вот как же в самое, который старак был из возму ились, что в сердцу., На что не столи выходительстве, что он с ними и прошила несчастное. Превослезномием по показать с вами, по моему восметить старика, прошел передобраться, к нашу подрожалось старый, когда в первое и слушыми не знаю, не за то вас случать, так, примерил вас, не совару и, что вы с просила меня, что я себя посмеются, что у ведь не могла от не становите. Откровит с свойму не страдникахо и тажется подавала отверяли, теперь солетную веденьку, по меня! Они вам подорвали. Вы с самие привосходило себя подавала на меня, чтоб я все подобное и воселается от нашего подоржнось на веснитьсе вашему все теперь! Ну, да вос перебрагела, то я все, когда он прежде серезния, потому что он полижу то вам ни служали и продавить в себя и не мое не в то 